# Galvo test notebook
In this notebook, you will find different use cases to use the Luos_engine Galvo example.
To make it work you need to have the Galvo controlling board connected as well as the custom gate contained in the [Product example code](https://github.com/Luos-io/luos_engine/tree/main/examples/projects/product).
By executing the next cell you should see your Galvo and Gate board, then you will be ready to use it.
If you have any questions about it please contact the [Luos_engine community on discord](http://bit.ly/JoinLuosDiscord)

In [ ]:
import time
import sys
from pyluos import Device, map_custom_service
from IPython.display import clear_output

# Import custom service and map it
sys.path.append('/Users/nicolasrabault/Projects/luos/luos_engine/examples/projects/product')
from point_2D import Point_2D
map_custom_service("point_2D", Point_2D)

device = Device('localhost') #/dev/cu.usbserial-D308N897
#device = Device('localhost', port=8000)
print(device.nodes)

## Send a simple point to reach

In [ ]:
device.galvo.position = (4000, 4000)
device.galvo.play()

## Send multiple points to reach

In [ ]:
# we reduce the sampling frequency to make it easy to see
device.galvo.sampling_freq = 100
device.galvo.position = [(0, 0), (20000,20000)]
device.galvo.play()

By default the Galvo is in single mode. This mean that your trajectory wil be played only one time.
Now your small trajectory is loaded into the memory so you can play it again by calling `device.galvo.play()`

In [ ]:
device.galvo.play()

Alternatively you can choose to switch in continuous mode to play your trajectory in loop.

In [ ]:
device.galvo.continuous()
device.galvo.play()

In [ ]:
device.galvo.stop()

# SVG demo 🎨
In this demo, we use an SVG input and display it.
First, execute the next cell to load the function then you will be able to play with it.

In [ ]:
from svg.path import parse_path
from xml.dom import minidom
import pygame
import numpy as np
import requests

# Create some functions to convert svg into trajectory
def get_point_at(path, distance, scale, offset):
    pos = path.point(distance)
    pos += offset
    pos *= scale
    return pos.real, pos.imag

def points_from_path(path, density, scale, offset):
    step = int(path.length() * density)
    last_step = step - 1

    if last_step <= 0:
        yield get_point_at(path, 0, scale, offset)
        return

    for distance in range(step):
        yield get_point_at(
            path, distance / last_step, scale, offset)

def points_from_svg(url, density=1, scale=1, offset=0):
    response = requests.get(url)
    if response.status_code == 200:
        svg_content = response.text
        doc = minidom.parseString(svg_content)
    else:
        print("Can't reach " + url)
        return

    clear_output(wait=True)
    start_time = time.time()
    offset = offset[0] + offset[1] * 1j
    points = []
    offsets = []
    i = 0
    for element in doc.getElementsByTagName("path"):
        for path in parse_path(element.getAttribute("d")):
            points.extend(points_from_path(
                path, density, scale, offset))
        i = i+1

    end_time = time.time()
    elapsed_time = end_time - start_time
    #print(f"Execution time: {elapsed_time} seconds")
    return points

def animated_svg_translation(url, density, scale, move_from, move_to, frame_nb):
    frames = []
    point1 = np.array(move_from)
    point2 = np.array(move_to)
    linear_points = np.linspace(point1, point2, frame_nb)
    for offset in linear_points:
        frames.append(points_from_svg(url, density=density, scale=scale, offset=(offset[0], offset[1])))
        print("Generating animation : " + str(offset[0]*100.0/move_to[0]) + "%")
    return frames

def play_animation(frames, scale):
    pygame.init()
    
    framerate = 15
    screen = pygame.display.set_mode([500, 500])
    screen.fill((255, 255, 255))
    running = True
    frame_id = 0
    device.galvo.continuous()
    device.galvo.position = frames[frame_id]
    time.sleep(0.1)
    device.galvo.play()
    while running:
        frame_id = frame_id + 1
        if (frame_id >= len(frames)):
            frame_id = 0
        screen.fill((255, 255, 255))
        for point in frames[frame_id]:
            point = tuple(ti/scale for ti in point)
            pygame.draw.circle(screen, (0, 0, 0), point, 1)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        device.galvo.position = frames[frame_id]
        time.sleep(1/framerate)
    pygame.quit()
    device.galvo.stop()

## Simply display a svg

In [ ]:
# set the sample freq up to properly display it
device.galvo.sampling_freq = 4000
# load an svg image
points = points_from_svg("https://mirrors.creativecommons.org/presskit/icons/cc.svg", density=0.3, scale=400, offset=(1, 100))
# we want to continuously display it
device.galvo.continuous()
device.galvo.position = points
# wait for the points to be sent before playing it
time.sleep(0.1)
device.galvo.play()

In [ ]:
device.galvo.stop()

## Generate an animation out of the svg and play it

In [ ]:
# compute a simple translation animation of an svg
frames = animated_svg_translation("https://mirrors.creativecommons.org/presskit/icons/cc.svg", density=0.3, scale=400, move_from=(0, 0), move_to=(90, 90), frame_nb=30)

In [ ]:
# play the previously generated animation
# you can close the pygame window to stop it
play_animation(frames, 100)

# Live audio FFT demo 🔊
In this demo, the function downloads a sound file plays it, and displays the FFT of the sound on the galvo.
First, execute the next cell to load the function then you will be able to play with it.

In [ ]:
from IPython.display import Audio
from IPython.display import clear_output
import time
import numpy as np

# Load an audio file play it and display the fft
# default file is:
#'Ignis' by Scott Buckley - released under CC-BY 4.0. www.scottbuckley.com.au
def live_display_fft(audio_file_link = 'https://www.scottbuckley.com.au/library/wp-content/uploads/2024/01/Ignis.mp3'):
    rate = 44100 #I don't know how to automatically get it from URL

    # Slow down the galvo speed 
    device.galvo.sampling_freq = 2000
    
    # Create an Audio object
    print("downloading the audio file ...")
    clear_output(wait=True)
    audio_stream = Audio(url=audio_file_link, embed=True, autoplay=True)
    
    # Enable the galvo
    fft_size = 64
    X_scale = 1000
    init_frame = []
    Y_offset = 500
    Y_scale = 1600
    FPS = 20
    for i in range(fft_size):
        init_frame.append((i*X_scale, Y_offset))
    
    device.galvo.continuous()
    device.galvo.position = init_frame
    time.sleep(0.1)
    device.galvo.play()
    
    # Display the audio
    display(audio_stream)
    
    # Live animation
    def chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))
    
    sample_per_frame = int(rate/FPS)
    for chunk in chunker(audio_stream.data, sample_per_frame):
        chunk = np.frombuffer(chunk, dtype=np.uint8)
        fft = abs(np.fft.fft(chunk, n=fft_size).real)
        # Create points out of it
        frame = []
        freq=0
        for ampl in fft:
            frame.append((freq, Y_offset+(ampl*Y_scale)))
            freq = freq + X_scale
        device.galvo.position = frame
        time.sleep(1/FPS)
    device.galvo.stop()
    # set the default sampling freq
    device.galvo.sampling_freq = 10000

## Live fft
By executing this function you will play a song and live display the fft on the galvo.
You can hit pause on the reader and stop the notebook to stop the function.
You will have to manually stop the galvo if you stop the function.

In [ ]:
live_display_fft()

In [ ]:
device.galvo.stop()